# Object Models

 * `OrbitState`: initial orbit state with OMM-based attributes as a component of the propagation analysis input
 * `PropagationRequest`: input to propagation analysis function
 * `PropagationRecord`: time, position, and velocity attributes as a component of the propagation analysis output
 * `PropagationResponse`: output from propagation analysis function

In [1]:
from datetime import datetime, timedelta
from typing import List, Optional

from pydantic import AwareDatetime, BaseModel, Field
from sgp4 import exporter, omm
from sgp4.api import Satrec

class OrbitState(BaseModel):
    object_name: Optional[str] = Field(None, description="Object name.")
    object_id: Optional[str] = Field(None, description="Object identifier.")
    epoch: datetime = Field(..., description="Epoch (time of orbit element specification).")
    mean_motion: float = Field(..., gt=0, description="Mean motion (revolutions per day).")
    eccentricity: float = Field(..., ge=0, lt=1, description="Eccentricity.")
    inclination: float = Field(..., ge=0, le=180, description="Inclination (degrees).")
    ra_of_asc_node: float = Field(..., ge=0, lt=360, description="Right ascension of ascending node (degrees).")
    arg_of_pericenter: float = Field(..., ge=0, lt=360, description="Argument of pericenter (degrees).")
    mean_anomaly: float = Field(..., ge=0, lt=360, description="Mean anomaly (degrees).")
    ephemeris_type: Optional[int] = Field(0, ge=0, description="Ephemeris type (0: Default (Kozai), 1: SGP, 2: SGP4 (Brouwer), 3: SDP4, 4: SGP8, 5: SDP8).")
    classification_type: Optional[str] = Field("U", pattern="[UCS]", description="Classification (U: unclassified, C: classified, S: secret)." )
    norad_cat_id: Optional[int] = Field(None, ge=0, description="NORAD catalog identifier.")
    element_set_no: Optional[int] = Field(None, ge=0, le=999, description="Element set number.")
    rev_at_epoch: Optional[int] = Field(None, ge=0, le=99999, description="Revolution number at epoch.")
    bstar: float = Field(0.0, ge=0, lt=1, description="B-star drag term (radiation pressure coefficient).")
    mean_motion_dot: float = Field(0.0, gt=-1, lt=1, description="First derivative of mean motion (ballistic coefficient).")
    mean_motion_ddot: float = Field(0.0, gt=-1, lt=1, description="Second derivative of mean motion.")

    @classmethod
    def from_omm(cls, omm) -> 'OrbitState':
        return OrbitState.model_validate(dict([(key.lower(), value) for key, value in omm.items()]))
    
    def to_omm(self) -> dict:
        return dict([
            (
                key.upper(), 
                "" if value is None
                else value.isoformat() if isinstance(value, datetime)
                else value
            )
            for key, value in self.model_dump().items()
        ])

    def to_satrec(self) -> Satrec:
        sat = Satrec()
        omm.initialize(sat, self.to_omm())
        return sat

    def to_tle(self) -> List[str]:
        return exporter.export_tle(self.to_satrec())

class PropagationRequest(BaseModel):
    orbit: OrbitState = Field(
        ...,
        description="Orbit to be propagated."
    )
    start: AwareDatetime = Field(
        ...,
        description="Propagation start time."
    )
    duration: timedelta = Field(
        ...,
        description="Propagation duration."
    )
    time_step: timedelta = Field(
        ...,
        description="Propagation time step duration."
    )

class PropagationRecord(BaseModel):
    time: datetime = Field(..., description="Time")
    position: List[float] = Field(..., min_length=3, max_length=3, description="Position")
    velocity: List[float] = Field(..., min_length=3, max_length=3, description="Velocity")

class PropagationResponse(BaseModel):
    records: List[PropagationRecord] = Field(
        [],
        description="Propagation results"
    )

# TAT-C Interface Implementation

Specifies a function with well-defined input (`PropagationRequest`) and output (`PropagationResponse`) using TAT-C's implementation.

In [2]:
from tatc.schemas import Instrument, Satellite, TwoLineElements
from tatc.analysis import collect_orbit_track, OrbitCoordinate, OrbitOutput

import pandas as pd

def propagate_tatc(request: PropagationRequest) -> PropagationResponse:
    return PropagationResponse(
        records=collect_orbit_track(
            Satellite(
                name=request.orbit.object_name,
                orbit=TwoLineElements(
                    tle=request.orbit.to_tle()
                )
            ),
            Instrument(name="Instrument"),
            pd.date_range(request.start, request.start + request.duration, freq=request.time_step),
            coordinates=OrbitCoordinate.ECI,
            orbit_output=OrbitOutput.POSITION_VELOCITY
        ).apply(lambda r: PropagationRecord(
            time = r.time,
            position = r.geometry.coords[0],
            velocity = r.velocity.coords[0]
        ), axis = 1)
    )

# Example

Pulls OMM file from Celestrak, issues propagation request starting Jan 1 2024 for 1 day (1 minute time step), and displays results.

Note that results are formatted as Python objects. The JSON-serialized outputs can be produced by replacing `model_dump` with `model_dump_json`; however, they do not display nicely in a Jupyter notebook.

In [3]:
import requests
import json
from datetime import datetime, timedelta, timezone

response = requests.get("https://celestrak.org/NORAD/elements/gp.php?NAME=ZARYA&FORMAT=JSON")
iss = json.loads(response.content)[0]

request = PropagationRequest(
    orbit = OrbitState.from_omm(iss),
    start = datetime(2024, 1, 1, tzinfo = timezone.utc),
    duration = timedelta(days=1),
    time_step = timedelta(minutes=1)
)

display(request.model_dump())

response = propagate_tatc(request)

display(response.model_dump())

{'orbit': {'object_name': 'ISS (ZARYA)',
  'object_id': '1998-067A',
  'epoch': datetime.datetime(2024, 6, 7, 16, 17, 46, 487328),
  'mean_motion': 15.50990455,
  'eccentricity': 0.0005858,
  'inclination': 51.6417,
  'ra_of_asc_node': 2.3964,
  'arg_of_pericenter': 284.8552,
  'mean_anomaly': 189.607,
  'ephemeris_type': 0,
  'classification_type': 'U',
  'norad_cat_id': 25544,
  'element_set_no': 999,
  'rev_at_epoch': 45707,
  'bstar': 0.0003511,
  'mean_motion_dot': 0.00020356,
  'mean_motion_ddot': 0.0},
 'start': datetime.datetime(2024, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'duration': datetime.timedelta(days=1),
 'time_step': datetime.timedelta(seconds=60)}

{'records': [{'time': Timestamp('2024-01-01 00:00:00+0000', tz='UTC'),
   'position': [-3507523.947019255, -5736901.046961996, 1078904.3796157178],
   'velocity': [3592.166748503531, -3318.0335255489613, -5887.382302806406]},
  {'time': Timestamp('2024-01-01 00:01:00+0000', tz='UTC'),
   'position': [-3284181.6007106104, -5922791.358651787, 723468.0353002548],
   'velocity': [3849.69261239933, -2875.924570964579, -5955.9175339311905]},
  {'time': Timestamp('2024-01-01 00:02:00+0000', tz='UTC'),
   'position': [-3045908.8050616872, -6081755.577584816, 364733.3457817332],
   'velocity': [4089.6581935371682, -2420.8478779579455, -5997.286201350921]},
  {'time': Timestamp('2024-01-01 00:03:00+0000', tz='UTC'),
   'position': [-2793792.2743444075, -6213078.004951221, 4336.229104260685],
   'velocity': [4310.974961590507, -1954.891107706065, -6011.312300539177]},
  {'time': Timestamp('2024-01-01 00:04:00+0000', tz='UTC'),
   'position': [-2528981.4367389884, -6316169.705201646, -356080.64823